<a href="https://colab.research.google.com/github/JhonFiUNFV/python_prep/blob/master/Spark_SQL_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.- Configuracion Entorno Spark

In [ ]:
# Descargamos spark con hadoop y Java
!apt-get update # Actualizar el gestor de paquetes de Linux
!apt-get install openjdk-8-jdk-headless -qq > /dev/null # Descargar Java
!wget -q https://dlcdn.apache.org/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz # Descargar Apache Spark
!tar xf spark-2.4.8-bin-hadoop2.7.tgz #Descomprimir Spark
!pip install -q findspark # Para Utilizar Spark

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (141 kB/s)
Reading package lists... Done


In [ ]:
# Seteamos las variables de entorno
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"

In [ ]:
# Creamos la conexion a Spark 
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
spark

# 2.- Importacion de Librerias

In [ ]:
# Librerias Spark
from pyspark.sql import functions as F

#Librerias Python
import pandas as pd

# 3.- Importacion de Datos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Poner la direcion de la ruta aqui
ruta = "/content/drive/MyDrive/DATA/Data_Estudiantes/"

## 3.1.- Base de Cursos

In [ ]:
# Importar el archivo "Curso.csv"
curso = spark.read.csv(ruta + 'Curso.csv', sep = ';', header = True, inferSchema = True)
curso.show()

+-------+----------------+-----+--------+------------+
|idCurso|          nombre|costo|duracion|idInstructor|
+-------+----------------+-----+--------+------------+
|      1|             SQL|  100|       2|           6|
|      2|          Python|   80|       5|           3|
|      3|               R|   70|       5|           2|
|      4|    SPSS Modeler|   50|       4|           1|
|      5|Machine Learning|  200|       8|           1|
|      6|        Big Data|  150|       6|           5|
|      7|         Tableau|   60|       2|           6|
|      8|           Linux|   75|       4|        null|
+-------+----------------+-----+--------+------------+



## 3.2.- Base de Estudiantes

In [ ]:
# Importar el archivo "Estudiantes.csv"
estudiantes = spark.read.csv(ruta + "Estudiantes.csv",
                             sep = '|', header = True, inferSchema = True)
estudiantes.show()

+-------------+-----------+----------+---------+
|codEstudiante|     nombre|  apellido| telefono|
+-------------+-----------+----------+---------+
|            1|Jean Pierre|    Levano|987654321|
|            2|      Sandy|  Carbajal|987654322|
|            3|   Josselin|     Taipe|987654323|
|            4|      James|Surichaqui|987654324|
|            5|     Javier|      Ruiz|987654325|
|            6|    Enrique|   Basauri|987654326|
|            7|  Alexander|  Broncano|987654327|
|            8|      Marco|     Marin|987654328|
|            9|      Oscar|    Urbano|987654329|
|           10|   Yankarlo|   Vilchez|987654330|
|           11|  Giancarlo|  Torrejon|987654331|
|           12|    Roberto|   Alvarez|987654332|
|           13|      Bruno|      Diaz|987654333|
+-------------+-----------+----------+---------+



## 3.1.- Base de Instructores

In [ ]:
# Importar el archivo "Instructor.txt"
instructor = spark.read.csv(ruta + "Instructor.txt", sep="\t", header=True,inferSchema=True)
instructor.show()

+------------+------+---------+---------+
|idInstructor|nombre| apellido| telefono|
+------------+------+---------+---------+
|           1| Andre|   Chavez|123456789|
|           2|  Jose| Cardenas|123456788|
|           3| Mario|Bocanegra|123456787|
|           4|Charli|  Gutarra|123456786|
|           5|  Aldo|   Chavez|123456785|
|           6| Frank|   Rivera|123456784|
|           7|Walter|   Pinedo|123456783|
+------------+------+---------+---------+



## 3.1.- Base de Matriculas

In [ ]:
# Importar el archivo "Matricula.txt"
matricula = spark.read.csv(ruta + "Matricula.txt",
                           sep="\t",header=True,inferSchema=True)
matricula.show(5)

+-------+------------+-------+----+------------+
|idCurso|idEstudiante|periodo|nota|idInstructor|
+-------+------------+-------+----+------------+
|      1|           1| 201801|   7|           6|
|      1|           2| 201801|   8|           6|
|      1|           3| 201801|   9|           6|
|      1|           4| 201801|   2|           6|
|      2|           1| 201801|   5|           3|
+-------+------------+-------+----+------------+
only showing top 5 rows



# 4.- Uso de Join

In [ ]:
SELECT A.*, B.*
FROM TABLA1 A
[INNER|LEFT|RIGTH|FULL] JOIN TABLA2 ON A.CLAVE1 = B.CLAVE2

## 4.1.- Inner Join

    // Inner join implicito
    Cuando las claves son iguales:
    df1.join(df2, on=["clave"])
    Cuando las claves no son iguales:
    df1.join(df2, df1("clave1") == df2("clave2"))
    
    // Inner join explicito
    Cuando las claves son iguales:
    df1.join(df2, on=["clave"], how='inner')
    Cuando las claves no son iguales:
    df1.join(df2, df1("clave1") == df2("clave2"), "inner")

In [ ]:
# SELECT A.*, B.*
# FROM CURSO A
# INNER JOIN INSTRUCTOR B
# ON A.idInstructor = B.idInstructor
# Forma 1
curso.join(instructor, on=["idInstructor"]).show()

+------------+-------+----------------+-----+--------+------+---------+---------+
|idInstructor|idCurso|          nombre|costo|duracion|nombre| apellido| telefono|
+------------+-------+----------------+-----+--------+------+---------+---------+
|           1|      5|Machine Learning|  200|       8| Andre|   Chavez|123456789|
|           1|      4|    SPSS Modeler|   50|       4| Andre|   Chavez|123456789|
|           2|      3|               R|   70|       5|  Jose| Cardenas|123456788|
|           3|      2|          Python|   80|       5| Mario|Bocanegra|123456787|
|           5|      6|        Big Data|  150|       6|  Aldo|   Chavez|123456785|
|           6|      7|         Tableau|   60|       2| Frank|   Rivera|123456784|
|           6|      1|             SQL|  100|       2| Frank|   Rivera|123456784|
+------------+-------+----------------+-----+--------+------+---------+---------+



In [ ]:
# Forma 2
curso.join(instructor, curso["idInstructor"] == instructor["idInstructor"]).show()

+-------+----------------+-----+--------+------------+------------+------+---------+---------+
|idCurso|          nombre|costo|duracion|idInstructor|idInstructor|nombre| apellido| telefono|
+-------+----------------+-----+--------+------------+------------+------+---------+---------+
|      5|Machine Learning|  200|       8|           1|           1| Andre|   Chavez|123456789|
|      4|    SPSS Modeler|   50|       4|           1|           1| Andre|   Chavez|123456789|
|      3|               R|   70|       5|           2|           2|  Jose| Cardenas|123456788|
|      2|          Python|   80|       5|           3|           3| Mario|Bocanegra|123456787|
|      6|        Big Data|  150|       6|           5|           5|  Aldo|   Chavez|123456785|
|      7|         Tableau|   60|       2|           6|           6| Frank|   Rivera|123456784|
|      1|             SQL|  100|       2|           6|           6| Frank|   Rivera|123456784|
+-------+----------------+-----+--------+---------

In [ ]:
# SELECT A.*, B.NOMBRE as NOMBRE_PROFESOR, B.apellido as APELLIDO_PROFESOR
# FROM curso A
# inner join instructor B
# on A.idInstructor = B.idInstructor
curso.alias('A').join(instructor.alias('B'), on=["idInstructor"])\
  .select(F.col('A.*'),
           F.col('B.nombre').alias('NOMBREPROFESOR'),
          F.col('B.apellido').alias('APELLIDO_PROFESOR') ).show()

+------------+-------+----------------+-----+--------+--------------+-----------------+
|idInstructor|idCurso|          nombre|costo|duracion|NOMBREPROFESOR|APELLIDO_PROFESOR|
+------------+-------+----------------+-----+--------+--------------+-----------------+
|           1|      5|Machine Learning|  200|       8|         Andre|           Chavez|
|           1|      4|    SPSS Modeler|   50|       4|         Andre|           Chavez|
|           2|      3|               R|   70|       5|          Jose|         Cardenas|
|           3|      2|          Python|   80|       5|         Mario|        Bocanegra|
|           5|      6|        Big Data|  150|       6|          Aldo|           Chavez|
|           6|      7|         Tableau|   60|       2|         Frank|           Rivera|
|           6|      1|             SQL|  100|       2|         Frank|           Rivera|
+------------+-------+----------------+-----+--------+--------------+-----------------+



In [ ]:
# SELECT A.nombre as NOMBRE_CURSO, B.NOMBRE as NOMBRE_PROFESOR
# FROM curso A
# inner join instructor B
# on A.idInstructor = B.idInstructor
curso.alias('A').join(instructor.alias('B'), F.col('A.idInstructor') == F.col('B.idInstructor') )\
  .select(F.col('A.nombre').alias('NOMBRE_CURSO'),
          F.col('B.nombre').alias('NOMBRE_PROFESOR') ).show()

+----------------+---------------+
|    NOMBRE_CURSO|NOMBRE_PROFESOR|
+----------------+---------------+
|             SQL|          Frank|
|          Python|          Mario|
|               R|           Jose|
|    SPSS Modeler|          Andre|
|Machine Learning|          Andre|
|        Big Data|           Aldo|
|         Tableau|          Frank|
+----------------+---------------+



**Ejercicio 1: Mostrar el nombre del curso, el costo y el nombre y apellido del instructor que lo dicta**

In [ ]:
# Resolver Aqui
curso.alias('A').join(instructor.alias('B'), on=["idInstructor"])\
  .select(F.col('A.nombre'),F.col('A.costo'),
           F.col('B.nombre').alias('Nombre_Instructor'),
          F.col('B.apellido').alias('Apellido_Instructor') ).show()

+----------------+-----+-----------------+-------------------+
|          nombre|costo|Nombre_Instructor|Apellido_Instructor|
+----------------+-----+-----------------+-------------------+
|Machine Learning|  200|            Andre|             Chavez|
|    SPSS Modeler|   50|            Andre|             Chavez|
|               R|   70|             Jose|           Cardenas|
|          Python|   80|            Mario|          Bocanegra|
|        Big Data|  150|             Aldo|             Chavez|
|         Tableau|   60|            Frank|             Rivera|
|             SQL|  100|            Frank|             Rivera|
+----------------+-----+-----------------+-------------------+



In [ ]:
curso.alias('A').join(instructor.alias('B'), F.col('A.idInstructor') == F.col('B.idInstructor') )\
  .select(F.col('A.nombre').alias('NOMBRE_CURSO'),
          F.col('A.costo').alias('COSTO'),
          F.col('B.nombre').alias('NOMBRE_PROFESOR'),
          F.col('B.apellido').alias('APELLIDO_PROFESOR'), ).show()

+----------------+-----+---------------+-----------------+
|    NOMBRE_CURSO|COSTO|NOMBRE_PROFESOR|APELLIDO_PROFESOR|
+----------------+-----+---------------+-----------------+
|Machine Learning|  200|          Andre|           Chavez|
|    SPSS Modeler|   50|          Andre|           Chavez|
|               R|   70|           Jose|         Cardenas|
|          Python|   80|          Mario|        Bocanegra|
|        Big Data|  150|           Aldo|           Chavez|
|         Tableau|   60|          Frank|           Rivera|
|             SQL|  100|          Frank|           Rivera|
+----------------+-----+---------------+-----------------+



In [ ]:
curso.alias('A').join(instructor.alias('B'), on=["idInstructor"])\
  .select(F.col('A.nombre').alias('NOMBRE_CURSO'),
          F.col('A.costo').alias('COSTO_CURSO'),
           F.col('B.nombre').alias('NOMBRE_INSTRUCTOR'),
          F.col('B.apellido').alias('APELLIDO_INSTRUCTOR') ).show()

+----------------+-----------+-----------------+-------------------+
|    NOMBRE_CURSO|COSTO_CURSO|NOMBRE_INSTRUCTOR|APELLIDO_INSTRUCTOR|
+----------------+-----------+-----------------+-------------------+
|Machine Learning|        200|            Andre|             Chavez|
|    SPSS Modeler|         50|            Andre|             Chavez|
|               R|         70|             Jose|           Cardenas|
|          Python|         80|            Mario|          Bocanegra|
|        Big Data|        150|             Aldo|             Chavez|
|         Tableau|         60|            Frank|             Rivera|
|             SQL|        100|            Frank|             Rivera|
+----------------+-----------+-----------------+-------------------+



In [ ]:
curso.alias('A').join(instructor.alias('B'), F.col('A.idInstructor') == F.col('B.idInstructor') )\
  .select(F.col('A.nombre').alias('NOMBRE_CURSO'),
          F.col('A.costo').alias('COSTO_CURSO'),
          F.col('B.nombre').alias('NOMBRE_PROFESOR'),
          F.col('B.apellido').alias('APELLIDO_PROFESOR')).show()

+----------------+-----------+---------------+-----------------+
|    NOMBRE_CURSO|COSTO_CURSO|NOMBRE_PROFESOR|APELLIDO_PROFESOR|
+----------------+-----------+---------------+-----------------+
|Machine Learning|        200|          Andre|           Chavez|
|    SPSS Modeler|         50|          Andre|           Chavez|
|               R|         70|           Jose|         Cardenas|
|          Python|         80|          Mario|        Bocanegra|
|        Big Data|        150|           Aldo|           Chavez|
|         Tableau|         60|          Frank|           Rivera|
|             SQL|        100|          Frank|           Rivera|
+----------------+-----------+---------------+-----------------+



## 4.2.- Left Join

    Cuando las claves son iguales:
    df1.join(df2, on=["clave"], how='left')
    Cuando las claves no son iguales:
    df1.join(df2, df1("clave1") == df2("clave2"), "left")

In [ ]:
# SELECT A.*, B.*
# FROM CURSO A
# LEFT JOIN INSTRUCTOR B
# ON A.idInstructor = B.idInstructor
# Forma 1
curso.join(instructor, on=["idInstructor"], how = 'left').show()

+------------+-------+----------------+-----+--------+------+---------+---------+
|idInstructor|idCurso|          nombre|costo|duracion|nombre| apellido| telefono|
+------------+-------+----------------+-----+--------+------+---------+---------+
|           6|      1|             SQL|  100|       2| Frank|   Rivera|123456784|
|           3|      2|          Python|   80|       5| Mario|Bocanegra|123456787|
|           2|      3|               R|   70|       5|  Jose| Cardenas|123456788|
|           1|      4|    SPSS Modeler|   50|       4| Andre|   Chavez|123456789|
|           1|      5|Machine Learning|  200|       8| Andre|   Chavez|123456789|
|           5|      6|        Big Data|  150|       6|  Aldo|   Chavez|123456785|
|           6|      7|         Tableau|   60|       2| Frank|   Rivera|123456784|
|        null|      8|           Linux|   75|       4|  null|     null|     null|
+------------+-------+----------------+-----+--------+------+---------+---------+



In [ ]:
# Forma 2
curso.join(instructor, curso["idInstructor"] == instructor["idInstructor"], how = 'left').show()

+-------+----------------+-----+--------+------------+------------+------+---------+---------+
|idCurso|          nombre|costo|duracion|idInstructor|idInstructor|nombre| apellido| telefono|
+-------+----------------+-----+--------+------------+------------+------+---------+---------+
|      1|             SQL|  100|       2|           6|           6| Frank|   Rivera|123456784|
|      2|          Python|   80|       5|           3|           3| Mario|Bocanegra|123456787|
|      3|               R|   70|       5|           2|           2|  Jose| Cardenas|123456788|
|      4|    SPSS Modeler|   50|       4|           1|           1| Andre|   Chavez|123456789|
|      5|Machine Learning|  200|       8|           1|           1| Andre|   Chavez|123456789|
|      6|        Big Data|  150|       6|           5|           5|  Aldo|   Chavez|123456785|
|      7|         Tableau|   60|       2|           6|           6| Frank|   Rivera|123456784|
|      8|           Linux|   75|       4|        n

In [ ]:
# SELECT A.*, B.NOMBRE as NOMBRE_PROFESOR, B.apellido as APELLIDO_PROFESOR
# FROM curso A
# LEFT JOIN instructor B
# ON A.idInstructor = B.idInstructor
curso.alias('A').join(instructor.alias('B'), on=["idInstructor"], how = 'left')\
  .select(F.col('A.*'),
           F.col('B.nombre').alias('NOMBREPROFESOR'),
          F.col('B.apellido').alias('APELLIDO_PROFESOR') ).show()

+------------+-------+----------------+-----+--------+--------------+-----------------+
|idInstructor|idCurso|          nombre|costo|duracion|NOMBREPROFESOR|APELLIDO_PROFESOR|
+------------+-------+----------------+-----+--------+--------------+-----------------+
|           6|      1|             SQL|  100|       2|         Frank|           Rivera|
|           3|      2|          Python|   80|       5|         Mario|        Bocanegra|
|           2|      3|               R|   70|       5|          Jose|         Cardenas|
|           1|      4|    SPSS Modeler|   50|       4|         Andre|           Chavez|
|           1|      5|Machine Learning|  200|       8|         Andre|           Chavez|
|           5|      6|        Big Data|  150|       6|          Aldo|           Chavez|
|           6|      7|         Tableau|   60|       2|         Frank|           Rivera|
|        null|      8|           Linux|   75|       4|          null|             null|
+------------+-------+----------

In [ ]:
# SELECT A.nombre as NOMBRE_CURSO, B.NOMBRE as NOMBRE_PROFESOR
# FROM curso A
# LEFT join instructor B
# ON A.idInstructor = B.idInstructor
curso.alias('A').join(instructor.alias('B'), F.col('A.idInstructor') == F.col('B.idInstructor'), how = 'left')\
  .select(F.col('A.nombre').alias('NOMBRE_CURSO'),
          F.col('B.nombre').alias('NOMBRE_PROFESOR') ).show()

+----------------+---------------+
|    NOMBRE_CURSO|NOMBRE_PROFESOR|
+----------------+---------------+
|             SQL|          Frank|
|          Python|          Mario|
|               R|           Jose|
|    SPSS Modeler|          Andre|
|Machine Learning|          Andre|
|        Big Data|           Aldo|
|         Tableau|          Frank|
|           Linux|           null|
+----------------+---------------+



**Pegar a la matricula el nombre de los cursos y el costo y mostrar cuantos matriculados hay por curso y el costo total**

In [ ]:
# Resolver Aqui
matricula.alias('A').join(curso.alias('B'), F.col('A.idCurso') == F.col('B.idCurso'), how = 'left')\
  .select(F.col('A.*'), 
          F.col('B.nombre').alias('NOMBRE_CURSO'), F.col('B.costo') )\
  .groupby('NOMBRE_CURSO').agg( F.count('*').alias('Alumnos_Matriculados'), F.sum('costo').alias('Costo_Total') ).show()

+----------------+--------------------+-----------+
|    NOMBRE_CURSO|Alumnos_Matriculados|Costo_Total|
+----------------+--------------------+-----------+
|Machine Learning|                  12|       2400|
|    SPSS Modeler|                   4|        200|
|               R|                   4|        280|
|        Big Data|                  12|       1800|
|             SQL|                  16|       1600|
|          Python|                   4|        320|
|         Tableau|                   4|        240|
+----------------+--------------------+-----------+



## 4.3.- Rigth Join

    Cuando las claves son iguales:
    df1.join(df2, on=["clave"], how='right')
    Cuando las claves no son iguales:
    df1.join(df2, df1("clave1") == df2("clave2"), "right_outer")

In [ ]:
# SELECT A.*, B.*
# FROM CURSO A
# LEFT JOIN INSTRUCTOR B
# ON A.idInstructor = B.idInstructor
# Forma 1
curso.join(instructor, on=["idInstructor"], how = 'right').show()

+------------+-------+----------------+-----+--------+------+---------+---------+
|idInstructor|idCurso|          nombre|costo|duracion|nombre| apellido| telefono|
+------------+-------+----------------+-----+--------+------+---------+---------+
|           1|      5|Machine Learning|  200|       8| Andre|   Chavez|123456789|
|           1|      4|    SPSS Modeler|   50|       4| Andre|   Chavez|123456789|
|           2|      3|               R|   70|       5|  Jose| Cardenas|123456788|
|           3|      2|          Python|   80|       5| Mario|Bocanegra|123456787|
|           4|   null|            null| null|    null|Charli|  Gutarra|123456786|
|           5|      6|        Big Data|  150|       6|  Aldo|   Chavez|123456785|
|           6|      7|         Tableau|   60|       2| Frank|   Rivera|123456784|
|           6|      1|             SQL|  100|       2| Frank|   Rivera|123456784|
|           7|   null|            null| null|    null|Walter|   Pinedo|123456783|
+------------+--

In [ ]:
instructor.join(curso, on=["idInstructor"], how = 'left').show()

+------------+------+---------+---------+-------+----------------+-----+--------+
|idInstructor|nombre| apellido| telefono|idCurso|          nombre|costo|duracion|
+------------+------+---------+---------+-------+----------------+-----+--------+
|           1| Andre|   Chavez|123456789|      5|Machine Learning|  200|       8|
|           1| Andre|   Chavez|123456789|      4|    SPSS Modeler|   50|       4|
|           2|  Jose| Cardenas|123456788|      3|               R|   70|       5|
|           3| Mario|Bocanegra|123456787|      2|          Python|   80|       5|
|           4|Charli|  Gutarra|123456786|   null|            null| null|    null|
|           5|  Aldo|   Chavez|123456785|      6|        Big Data|  150|       6|
|           6| Frank|   Rivera|123456784|      7|         Tableau|   60|       2|
|           6| Frank|   Rivera|123456784|      1|             SQL|  100|       2|
|           7|Walter|   Pinedo|123456783|   null|            null| null|    null|
+------------+--

In [ ]:
# Forma 2
curso.join(instructor, curso["idInstructor"] == instructor["idInstructor"], how = 'right').show()

+-------+----------------+-----+--------+------------+------------+------+---------+---------+
|idCurso|          nombre|costo|duracion|idInstructor|idInstructor|nombre| apellido| telefono|
+-------+----------------+-----+--------+------------+------------+------+---------+---------+
|      5|Machine Learning|  200|       8|           1|           1| Andre|   Chavez|123456789|
|      4|    SPSS Modeler|   50|       4|           1|           1| Andre|   Chavez|123456789|
|      3|               R|   70|       5|           2|           2|  Jose| Cardenas|123456788|
|      2|          Python|   80|       5|           3|           3| Mario|Bocanegra|123456787|
|   null|            null| null|    null|        null|           4|Charli|  Gutarra|123456786|
|      6|        Big Data|  150|       6|           5|           5|  Aldo|   Chavez|123456785|
|      7|         Tableau|   60|       2|           6|           6| Frank|   Rivera|123456784|
|      1|             SQL|  100|       2|         

## 4.4.- Full Join

    Cuando las claves son iguales:
    df1.join(df2, on=["clave"], how='full_outer')
    Cuando las claves no son iguales:
    df1.join(df2, df1("clave1") == df2("clave2"), "full")

In [ ]:
# SELECT A.*, B.*
# FROM CURSO A
# LEFT JOIN INSTRUCTOR B
# ON A.idInstructor = B.idInstructor
# Forma 1
curso.join(instructor, on=["idInstructor"], how = 'full').show()

+------------+-------+----------------+-----+--------+------+---------+---------+
|idInstructor|idCurso|          nombre|costo|duracion|nombre| apellido| telefono|
+------------+-------+----------------+-----+--------+------+---------+---------+
|        null|      8|           Linux|   75|       4|  null|     null|     null|
|           1|      4|    SPSS Modeler|   50|       4| Andre|   Chavez|123456789|
|           1|      5|Machine Learning|  200|       8| Andre|   Chavez|123456789|
|           6|      1|             SQL|  100|       2| Frank|   Rivera|123456784|
|           6|      7|         Tableau|   60|       2| Frank|   Rivera|123456784|
|           3|      2|          Python|   80|       5| Mario|Bocanegra|123456787|
|           5|      6|        Big Data|  150|       6|  Aldo|   Chavez|123456785|
|           4|   null|            null| null|    null|Charli|  Gutarra|123456786|
|           7|   null|            null| null|    null|Walter|   Pinedo|123456783|
|           2|  

In [ ]:
# Forma 2
curso.join(instructor, curso["idInstructor"] == instructor["idInstructor"], how = 'full').show()

+-------+----------------+-----+--------+------------+------------+------+---------+---------+
|idCurso|          nombre|costo|duracion|idInstructor|idInstructor|nombre| apellido| telefono|
+-------+----------------+-----+--------+------------+------------+------+---------+---------+
|      8|           Linux|   75|       4|        null|        null|  null|     null|     null|
|      4|    SPSS Modeler|   50|       4|           1|           1| Andre|   Chavez|123456789|
|      5|Machine Learning|  200|       8|           1|           1| Andre|   Chavez|123456789|
|      1|             SQL|  100|       2|           6|           6| Frank|   Rivera|123456784|
|      7|         Tableau|   60|       2|           6|           6| Frank|   Rivera|123456784|
|      2|          Python|   80|       5|           3|           3| Mario|Bocanegra|123456787|
|      6|        Big Data|  150|       6|           5|           5|  Aldo|   Chavez|123456785|
|   null|            null| null|    null|        n

In [ ]:
# SELECT A.*, B.NOMBRE as NOMBRE_PROFESOR, B.apellido as APELLIDO_PROFESOR
# FROM curso A
# LEFT JOIN instructor B
# ON A.idInstructor = B.idInstructor
curso.alias('A').join(instructor.alias('B'), on=["idInstructor"], how = 'full')\
  .select(F.col('A.*'),
           F.col('B.nombre').alias('NOMBREPROFESOR'),
          F.col('B.apellido').alias('APELLIDO_PROFESOR') ).show()

+-------+----------------+-----+--------+--------------+-----------------+
|idCurso|          nombre|costo|duracion|NOMBREPROFESOR|APELLIDO_PROFESOR|
+-------+----------------+-----+--------+--------------+-----------------+
|      8|           Linux|   75|       4|          null|             null|
|      4|    SPSS Modeler|   50|       4|         Andre|           Chavez|
|      5|Machine Learning|  200|       8|         Andre|           Chavez|
|      1|             SQL|  100|       2|         Frank|           Rivera|
|      7|         Tableau|   60|       2|         Frank|           Rivera|
|      2|          Python|   80|       5|         Mario|        Bocanegra|
|      6|        Big Data|  150|       6|          Aldo|           Chavez|
|   null|            null| null|    null|        Charli|          Gutarra|
|   null|            null| null|    null|        Walter|           Pinedo|
|      3|               R|   70|       5|          Jose|         Cardenas|
+-------+----------------

**Pegar a las matriculas los datos de los instructores, los estudiantes y despues sacar un reporte por curso y periodo para saber la nota promedio, la cantidad de alumnos y la suma de costos**

In [ ]:
matricula.show()

+-------+------------+-------+----+------------+
|idCurso|idEstudiante|periodo|nota|idInstructor|
+-------+------------+-------+----+------------+
|      1|           1| 201801|   7|           6|
|      1|           2| 201801|   8|           6|
|      1|           3| 201801|   9|           6|
|      1|           4| 201801|   2|           6|
|      2|           1| 201801|   5|           3|
|      2|           2| 201801|  17|           3|
|      2|           3| 201801|  11|           3|
|      2|           4| 201801|  18|           3|
|      3|           1| 201801|   3|           2|
|      3|           2| 201801|   2|           2|
|      3|           3| 201801|   7|           2|
|      3|           4| 201801|  14|           2|
|      4|           1| 201801|   0|           1|
|      4|           2| 201801|   3|           1|
|      4|           3| 201801|  10|           1|
|      4|           4| 201801|  19|           1|
|      1|           5| 201801|   0|           6|
|      1|           

In [ ]:
#Resolver Aqui
matricula.alias('A').join(curso.alias('B'), F.col('A.idCurso') == F.col('B.idCurso'), how = 'left')\
  .join(estudiantes.alias('C'), F.col('A.idEstudiante') == F.col('C.codEstudiante'), how = 'left')\
  .join(instructor.alias('D'), F.col('A.idInstructor') == F.col('D.idInstructor'), how = 'left')\
  .select(F.col('*'))\
  .groupby('B.nombre','periodo').agg( F.avg('nota').alias('Nota_Promedio'),
                                     F.count('*').alias('Cantidad_Alumnos'),
                                     F.sum('costo').alias('Suma_Costos') ).show()

+----------------+-------+-------------+----------------+-----------+
|          nombre|periodo|Nota_Promedio|Cantidad_Alumnos|Suma_Costos|
+----------------+-------+-------------+----------------+-----------+
|        Big Data| 201901|          8.5|               8|       1200|
|    SPSS Modeler| 201801|          8.0|               4|        200|
|             SQL| 201801|        7.125|               8|        800|
|Machine Learning| 201801|         4.75|               4|        800|
|        Big Data| 201801|        10.25|               4|        600|
|         Tableau| 201901|        12.75|               4|        240|
|          Python| 201801|        12.75|               4|        320|
|             SQL| 201901|       10.625|               8|        800|
|               R| 201801|          6.5|               4|        280|
|Machine Learning| 201901|       12.375|               8|       1600|
+----------------+-------+-------------+----------------+-----------+



In [ ]:
matricula.alias('A').join(curso.alias('B'), F.col('A.idCurso') == F.col('B.idCurso'), how = 'left')\
  .join(estudiantes.alias('C'), F.col('A.idEstudiante') == F.col('C.codEstudiante'), how = 'left')\
  .join(instructor.alias('D'), F.col('A.idInstructor') == F.col('D.idInstructor'), how = 'left')\
  .select(F.col('*'))\
  .groupby('B.nombre','periodo').agg( F.avg('nota').alias('Nota_matriculados'),F.count('*').alias('Alumnos_Matriculados'), F.sum('costo').alias('Costo_Total') ).show()

+----------------+-------+-----------------+--------------------+-----------+
|          nombre|periodo|Nota_matriculados|Alumnos_Matriculados|Costo_Total|
+----------------+-------+-----------------+--------------------+-----------+
|        Big Data| 201901|              8.5|                   8|       1200|
|    SPSS Modeler| 201801|              8.0|                   4|        200|
|             SQL| 201801|            7.125|                   8|        800|
|Machine Learning| 201801|             4.75|                   4|        800|
|        Big Data| 201801|            10.25|                   4|        600|
|         Tableau| 201901|            12.75|                   4|        240|
|          Python| 201801|            12.75|                   4|        320|
|             SQL| 201901|           10.625|                   8|        800|
|               R| 201801|              6.5|                   4|        280|
|Machine Learning| 201901|           12.375|                   8

## 4.5.- Trabajando un dataframe como Tabla de SQL

In [ ]:
instructor.createOrReplaceTempView("Tabla1")

In [ ]:
spark.sql("Select * from Tabla1 where nombre = 'Andre'").show()

+------------+------+--------+---------+
|idInstructor|nombre|apellido| telefono|
+------------+------+--------+---------+
|           1| Andre|  Chavez|123456789|
+------------+------+--------+---------+

